In [ ]:
%%time

import bempp.api
import numpy as np

import sys
sys.path.append('../../')
from decimal import *

import bemppUQ
from bemppUQ.operators.maxwell import assemble_operators, evaluate_far_field, evaluate_far_field_sd
from bemppUQ.shapes import kite, perturbate

bempp.api.global_parameters.assembly.potential_operator_assembly_type = 'dense'
bempp.api.global_parameters.hmat.eps = 1E-4

config = bemppUQ.config.set_case('A')
precision0 = 20
precision1 = 20

h0 = 2.0 * np.pi / (precision0 * config["k_int"])
h1 = 2.0 * np.pi / (precision1 * config["k_int"])

grid0 = kite(h=h0) 
grid1 = kite(h=h1) 

print(grid0.leaf_view.entity_count(1) * 2, 'NDOF')
print(grid1.leaf_view.entity_count(1) * 2, 'NDOF')

grid_eta0, grid_fun0 = perturbate(grid0, 0)
grid_eta1, grid_fun1 = perturbate(grid1, 0)

print('0')
transmission_operators0 = assemble_operators(grid0, config)
print('1')
transmission_operators1 = assemble_operators(grid1, config)
print('2')
Umean0, solution0 = evaluate_far_field(transmission_operators0, config)
print('3')
Umean1, solution1 = evaluate_far_field(transmission_operators1, config)
print('4')
result0 = evaluate_far_field_sd(grid0, transmission_operators0, config, solution0, grid_fun0, solve = True)
print('5')
result1 = evaluate_far_field_sd(grid1, transmission_operators1, config, solution1, grid_fun1, solve = True) 

In [ ]:
output = {}
output['result0'] = result0
output['result1'] = result1
output['Umean0'] = Umean0
output['Umean1'] = Umean1

import pickle
with open('results/full/output_full.txt', "wb") as fp:
    pickle.dump(output, fp)